Mareena Mallory 
July 13th, 2019

Problem Overview
------------------------------------------------------------------

Your client, The New York City, needs a better understanding of Citi Bike ridership. They have some questions about the Citi Bike performance in January 2017 and ask for your help.
The historical data for 2017 Jan is given to you.

The data includes: 
•	Trip Duration (seconds) 
•	Start Time and Date 
•	Stop Time and Date 
•	Start Station Name 
•	End Station Name 
•	Station ID 
•	Station Lat/Long
•	Bike ID 
•	User Type (Customer = 24-hour pass or 3-day pass user; Subscriber = Annual Member) 
•	Gender (Zero=unknown; 1=male; 2=female) 
•	Year of Birth 

The client wants to know the answers for the following questions, and it will be great if you can show some graphs to illustrate these problems

In [ ]:
# Importing and loading the data

import pandas as pd
import math
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import mpu


data = pd.read_csv('201701_citibike_tripdata.txt', sep=',')

In [ ]:
# Taking a peek at the first few rows
data.head()

# Looking at the overall statistics of the data
#data.describe(include='all')

# Seeing how many null values are present for each attribute (most are for User Type and Birth Year)
#print(data.isnull().sum())
#print(data.size)

In [ ]:
# In the case where a user takes a broken bike and immediately returns it, I will remove any cases where 
# the start station is the end station and the trip duration is less than 90 seconds (a somewhat arbitrary selection)
# based on how long it would take to return the bike after realizing it wasn't working properly
data = data.drop(data.index[(data['Start Station Latitude'] == data['End Station Latitude']) 
                            & (data['Trip Duration'] < 90) ])

Question 1
------------------------------------------------------------------
Top 5 stations with the most starts (showing # of starts)

In [ ]:
# Adding the counts of each station to two lists, one with the names and a second with the counts
data_counts = data['Start Station Name'].value_counts()
top_5_ID = data_counts.index[0:5].to_list()
top_5_counts = data_counts.head().to_list()

# Adding both the station name and count columns to a data frame for plotting
top_5_stations = pd.DataFrame()
top_5_stations['Start Station Name'] = top_5_ID
top_5_stations['Count'] = top_5_counts


print(top_5_stations)

In [ ]:
# Visualizing the top 5 as a bar graph
ax = top_5_stations.plot.bar(x='Start Station Name', y='Count', rot=0)
ax.set_title('Top 5 Bike Stations by Number of Starts', fontsize = 20)
plt.rcParams['figure.figsize'] = 20,7
plt.show()

Question 2
------------------------------------------------------------------
Trip duration by user type (Assuming to be average)

In [ ]:
# Grouping trip duration by user type and calcualting the average
trip_by_user = pd.DataFrame()

trip_by_user['Average Trip Duration'] = data.groupby('User Type')['Trip Duration'].mean()
trip_by_user = trip_by_user.reset_index()
trip_by_user['User Type'] = trip_by_user['User Type'].astype('object')

trip_by_user

In [ ]:
# Visualizing this with a box plot to look for potential anomalies
data.boxplot('Trip Duration', by = 'User Type')
plt.show()

In [ ]:
# Looks like their may be some outliers for trip duration. After further research, it seems that a trip longer 
# than 2 hours is unlikely as members get the first 45 included and non-members get the first 30 mintues included.
# Will be assuming that any trip over 2 hours or 7,200 seconds will be excluded
data = data.drop(data.index[(data['Trip Duration'] > 7200)])

In [ ]:
# Visualizing this with a box plot with outliers removed (This looks better)
data.boxplot('Trip Duration', by = 'User Type',showfliers=False)
plt.show()

It can be seen that the customers have, on average, longer trip durations than the subscribers

In [ ]:
# Re-grouping trip duration by user type and calculating the average
trip_by_user_clean = pd.DataFrame()

trip_by_user_clean['Average Trip Duration'] = data.groupby('User Type')['Trip Duration'].mean()
trip_by_user_clean = trip_by_user_clean.reset_index()
trip_by_user_clean['User Type'] = trip_by_user_clean['User Type'].astype('object')

trip_by_user_clean


In [ ]:
# Creating a bar chart to compare both trip durations by user
ax2 = trip_by_user_clean.plot.bar(x='User Type', y='Average Trip Duration', rot=0)
ax2.set_title('Average Trip Duration by User Type', fontsize = 20)
plt.rcParams['figure.figsize'] = 20,7
plt.show()


Question 3
------------------------------------------------------------------
Most popular trips based on start station and stop station

In [ ]:
# Finding the top 5 most popular trips
popular_trips = pd.DataFrame()

# Grouping by start name, then by end name
popular_trips = data.groupby(['Start Station Name','End Station Name']).size().reset_index(name = 'Number of Trips')
popular_trips['Start Station Name'] = popular_trips['Start Station Name'].astype(str)
popular_trips['End Station Name'] = popular_trips['End Station Name'].astype(str)

# Combining the start and end names to title the trip, and dropping the separate names from the tables for plotting 
popular_trips['Trip'] = popular_trips['Start Station Name'] + ' To ' + popular_trips['End Station Name']
popular_trips = popular_trips.drop(['Start Station Name', "End Station Name"], axis=1)

popular_trips = popular_trips.sort_values(by=['Number of Trips'], ascending=False)

top_5_popular_trips = popular_trips.head()

top_5_popular_trips

In [ ]:
# Creating a bar chart find the most popular trips
ax3 = top_5_popular_trips.plot.bar(x='Trip', y='Number of Trips', rot=90, fontsize = 15)
ax3.set_title('Most Popular Trips', fontsize = 20)
plt.rcParams['figure.figsize'] = 20,7
plt.show()

Question 4
------------------------------------------------------------------
Rider performance by Gender and Age based on avg trip distance (station to station), median speed (distance traveled / trip duration)

In [ ]:
# Starting by removing birth year missing data as it's a small portion of the data (alt: can fill with avgs) and 
# calculating the age of each rider for reference
data_clean = data.dropna(subset=['Birth Year']) 
#print (data_clean.isnull().sum())

# Calculation of age column
data_clean['Age'] = 2019 - data_clean['Birth Year']

# Assuming that age above 95 was a mistake
data_clean = data_clean.drop(data_clean.index[(data_clean['Age'] > 70)])


In [ ]:
# A function to calculate the Haversine Distance as it is less sensitive to round-off error than other methods
# that can occur when measuring distances that are located close together (as in this problem)
def Haversine(lat1,lon1,lat2,lon2, **kwarg):
    """
    formula:    a = sin²(Δφ/2) + cos φ1 ⋅ cos φ2 ⋅ sin²(Δλ/2)
    c = 2 ⋅ atan2( √a, √(1−a) )
    d = R ⋅ c
    where   φ is latitude, λ is longitude, R is earth’s radius (mean radius = 6,371km);
    note that angles need to be in radians to pass to trig functions!
    """
    R = 6371.0088
    lat1,lon1,lat2,lon2 = map(np.radians, [lat1,lon1,lat2,lon2])

    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2) **2
    c = 2 * np.arctan2(a**0.5, (1-a)**0.5)
    d = R * c
    return round(d,4)

# Calculation of the Distance column (haversine distance is being used in this case)
data_clean['Distance']  = Haversine(data_clean['Start Station Latitude'], data_clean['Start Station Longitude'],
                                    data_clean['End Station Latitude'], data_clean['End Station Longitude'])

In [ ]:
# Next we need to address the performance part of the question (median speed)
# Calculating how many hours each trip was

# For calculatiion ensuring the columns are of the right type
data_clean['TD_Minutes'] = data_clean['Trip Duration']/60

data_clean['min_per_mile'] = round(data_clean['TD_Minutes']/data_clean['Distance'], 2)

#Converting to miles per hour
data_clean['mile_per_hour'] = round(data_clean['Distance']/(data_clean['min_per_mile']/60),2)

data_clean.describe()

In [ ]:
# Research has suggested that trips longer than 30 miles may indicate the bike has been taken for repair, 
# these will be removed
data_clean = data_clean.drop(data_clean.index[(data_clean['Distance'] >= 30)])


# Knowing that the world record for bike speed is about 90 mile/hour we will drop anything above 50 mile/hour
data_clean = data_clean.drop(data_clean.index[(data_clean['mile_per_hour'] >= 50)])


# Removing any trips of duration 0
data_clean = data_clean.drop(data_clean.index[(data_clean['Distance'] == 0)])


# Also removing unknown gender 
data_clean = data_clean.drop(data_clean.index[(data_clean['Gender'] == 0)])
                          
data_clean.describe()

In [ ]:
# Displaying  performance by age and gender in minutes per mile
fig, ax4 = plt.subplots(figsize=(11,5))
data_clean.groupby(['Age','Gender']).median()['min_per_mile'].unstack().plot(ax=ax4, color ="rb")
ax4.legend(['Female','Male'])
plt.ylabel('Median Speed (Min/Mile)')
plt.title('Rider Performance Based on Gender and Age (Median Speed in Min/Mile)')
plt.show()

Question 5
------------------------------------------------------------------
What is the busiest bike in NYC in 2017? How many times was it used? How many minutes was it in use?

In [ ]:
# Finding the busiest bike by number of times used
bike_use = data_clean.groupby(['Bike ID']).size().reset_index(name = 'Number of Times Used');
bike_use = bike_use.sort_values('Number of Times Used', ascending = False);

# The busiest bike is ID 25147 used 34 times
top_5_bike_use = bike_use.head()
top_5_bike_use


In [ ]:
# plotting the top 5 most used bikes
ax5 = top_5_bike_use.plot.bar(x='Bike ID', y='Number of Times Used', rot=0, fontsize = 15)
ax5.set_title('Most Popular Bikes by Number of Times Used', fontsize = 20)
plt.rcParams['figure.figsize'] = 20,7
plt.show()

In [ ]:
# Finding the amount of minutes the top 5 most used bikes were used
bike_min = pd.DataFrame()
bike_min['Minutes Used'] = data_clean.groupby('Bike ID')['TD_Minutes'].sum().round(2)
bike_min = bike_min.sort_values('Minutes Used', ascending = False)
top_5_bike_min = bike_min.head()
top_5_bike_min

The top used bike was ID 27286. It was used 34 and for 357.48 minutes